In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from unidecode import unidecode

%matplotlib notebook

In [98]:
df = pd.read_csv('../data/preprocessed/details_home_for_sale_dataset_2019-07-25.csv')
df.head(5)


In [103]:
df_obj = (df[columns_to_analyze]
          .select_dtypes(include='object')
          .fillna('')
          .apply(lambda x: x.str.lower().apply(lambda y: unidecode(y)))
         )
df_num = df.select_dtypes(exclude='object')
df_raw = pd.concat([df_num, df_obj], axis=1)

In [102]:
columns_to_analyze = [
    'ano_de_construccion', 'aptos_por_piso', 'banos', 'banos_extra', 'cantidad_de_pisos',
    'description', 'disposicion', 'distancia_al_mar', 'dormitorios', 'dormitorios_extra',
    'estado', 'extra', 'garajes', 'garajes_extra', 'gastos_comunes', 'gastos_comunes_cur',
    'kind', 'm2_de_la_terraza', 'm2_del_terreno', 'm2_edificados', 'oficina', 'penthouse',
    'piso', 'plantas', 'plantas_extra', 'price', 'price_cur', 'sobre', 'tipo_propiedad',
    'vista_al_mar', 'vivienda_social', 'zona', 
]

In [4]:
mask = (
    df_raw['tipo_propiedad'].isin(['apartamentos', 'casas'])
    & (df_raw['oficina'] != 1)
    & (df_raw['penthouse'] != 1) 
)
df_raw = df_raw[mask]
df_raw.columns

Index(['acepta_permuta', 'altura_permitida', 'ambientes', 'ambientes_extra',
       'ano_de_construccion', 'aptos_por_piso', 'banos', 'banos_extra',
       'cantidad_de_pisos', 'casco', 'distancia_al_mar', 'dormitorios',
       'dormitorios_extra', 'financia', 'garajes', 'garajes_extra',
       'gastos_comunes', 'gastos_comunes_cur', 'hectareas', 'huespedes',
       'longitud_frente', 'm2_de_la_terraza', 'm2_del_terreno',
       'm2_edificados', 'oficina', 'penthouse', 'piso', 'plantas',
       'plantas_extra', 'price', 'price_cur', 'vista_al_mar',
       'vivienda_social', 'agency', 'description', 'direccion', 'disposicion',
       'estado', 'extra', 'kind', 'referencia', 'sobre', 'tipo_propiedad',
       'title', 'url', 'zona'],
      dtype='object')

In [6]:
columns = ['ano_de_construccion', 'banos', 'disposicion', 'dormitorios',
           'estado', 'garajes', 'gastos_comunes',
           'm2_de_la_terraza', 'm2_del_terreno', 'm2_edificados',
           'price', 'tipo_propiedad', 'vivienda_social', 'zona']

In [7]:
df_selected = df_raw[columns].reset_index(drop=True)
fill_zero_col = ['m2_de_la_terraza', 'vivienda_social', 'gastos_comunes', 'garajes']
df_selected.loc[:, fill_zero_col] = df_selected.loc[:, fill_zero_col].fillna(0)
mask_m2_terreno = df_selected['m2_del_terreno'].isna()
df_selected.loc[mask_m2_terreno, 'm2_del_terreno'] = df_selected.loc[mask_m2_terreno, 'm2_edificados']
(
    df_selected
    .isna()
    .sum()
)

ano_de_construccion    7664
banos                    73
disposicion               0
dormitorios              33
estado                    0
garajes                   0
gastos_comunes            0
m2_de_la_terraza          0
m2_del_terreno          827
m2_edificados           934
price                    70
tipo_propiedad            0
vivienda_social           0
zona                      0
dtype: int64

In [8]:
df_selected.dropna(inplace=True)

In [9]:
df_num_selected = df_selected.select_dtypes(exclude='object')
df_obj_selected = df_selected.select_dtypes(include='object')

In [10]:
df_obj_selected.sample(5)

,disposicion,estado,tipo_propiedad,zona
17,al frente,buen estado,casas,malvin norte
14109,al frente,buen estado,apartamentos,cordon
18980,al frente,excelente estado,apartamentos,tres cruces
17602,contrafrente,en construccion,apartamentos,malvin
18356,lateral,a estrenar,apartamentos,prado


In [11]:
zone_codes = pd.get_dummies(df_obj_selected['zona'])
property_type = pd.get_dummies(df_obj_selected['tipo_propiedad'])
property_state = pd.get_dummies(df_obj_selected['estado'])

In [12]:
zone_codes.columns = ['ZN_{}'.format(k.replace(' ', '_')) for k in zone_codes.columns]
property_state.columns = ['ST_{}'.format(k.replace(' ', '_')) for k in property_state.columns]


In [13]:
features_selected = pd.concat([df_num_selected, zone_codes, property_type, property_state], axis=1)
features_selected.to_csv('../data/ready/dataset_houses.csv', index=False)

In [14]:
price =  features_selected['price']
features = features_selected.drop(columns=['price'])



# Modeling

In [15]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import Normalizer

In [16]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [17]:
# scaler =  RobustScaler(quantile_range=(25, 75))
scaler = QuantileTransformer(output_distribution='normal') 
# scaler = PowerTransformer(method='box-cox')  # Non Linear | Strictly positive data
# scaler = PowerTransformer(method='yeo-johnson')  # Non Linear 



In [18]:
train_features, test_features = train_test_split(features, test_size=0.2, random_state=14)
train_price, test_price = train_test_split(price, test_size=0.2, random_state=14)

scaler.fit(train_features)
sca_train = scaler.transform(train_features)
sca_test = scaler.transform(train_features)

In [19]:
pca = PCA(n_components=.95, )
pca.fit(sca_train)
arr_train = pca.transform(sca_train)
arr_test = pca.transform(sca_test)

In [20]:
pca.explained_variance_ratio_

array([0.19129634, 0.13424812, 0.09001316, 0.06551298, 0.05066715,
       0.03811775, 0.02916101, 0.02800834, 0.02630605, 0.0257349 ,
       0.02334278, 0.02192643, 0.01921936, 0.01851215, 0.01614228,
       0.01487896, 0.01345692, 0.01225737, 0.01193237, 0.01178248,
       0.01106522, 0.01022184, 0.00965648, 0.00847697, 0.0075191 ,
       0.00669759, 0.00634692, 0.00623742, 0.00540183, 0.00534719,
       0.00522703, 0.00454237, 0.00414392, 0.0040206 , 0.00376112,
       0.00345288, 0.00341175, 0.00313883])

In [21]:
sns.pairplot(pd.DataFrame(arr[:, 0:4]))

NameError: name 'arr' is not defined

In [ ]:
# color = features.loc[features.index, 'zona'].astype('category').cat.codes
color = train_features.iloc[:, 1]
f, ax = plt.subplots(2,2)
ax[0, 0].scatter(arr[:,0], arr[:,1], c=color)
ax[0, 1].scatter(arr[:,1], arr[:,2], c=color)
ax[1, 0].scatter(arr[:,0], arr[:,2], c=color)
ax[1, 1].scatter(arr[:,3], arr[:,5], c=color)

In [ ]:
lda = LinearDiscriminantAnalysis(n_components=3).fit_transform(scaler.fit_transform(train_features),
                                                              train_price)

In [ ]:
color = train_features.iloc[:, 1]

f, ax = plt.subplots(2, 2)
ax[0, 0].scatter(lda[:, 0], lda[:, 1], c=color)
ax[0, 1].scatter(lda[:, 0], lda[:, 2], c=color)
ax[1, 0].scatter(lda[:, 1], lda[:, 2], c=color)


In [ ]:
def plot_predict_result(test_price, predict_price):
    f, ax = plt.subplots(2)
    ax[0].scatter(test_price, predict_price)
    ax[0].plot([0, 1600000], [0, 1600000], c='r')
    ax[1].hist(test_price - predict_price, bins=100)
    return f, ax

def get_scores(test_train, test_predict):
    mse = mean_squared_error(test_train, test_predict)
    mea = mean_absolute_error(test_train, test_predict)
    r2 = r2_score(test_train, test_predict)
    return 'mse={}, mea={}, r2={}'.format(mse, mea, r2)

# Linear Model

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(arr_train, train_price)
#model.coef_, model.intercept_, model.rank_

In [ ]:
predict_price = model.predict(arr_test)
get_scores(test_price, predict_price)

In [ ]:
f, ax = plot_predict_result(test_price, predict_price)
ax[0].set(ylim=[-10000000000, 10000000000])

# Regression Tree

In [ ]:
cls_tree = DecisionTreeRegressor()
cls_tree.fit(train_features, train_price)


In [ ]:
predic_price_tree = cls_tree.predict(test_features)
get_scores(predic_price_tree, predict_price)

In [ ]:
plot_predict_result(test_price, predic_price_tree)

In [ ]:
f, ax = plt.subplots()
(pd
 .Series(cls_tree.feature_importances_, index=train_features.columns)
 .sort_values()[-10:]
 .plot(ax=ax, kind='barh')
)

In [31]:
df_raw[df_raw['gastos_comunes']>1000000]
#df_selected['garajes'].unique()

,acepta_permuta,altura_permitida,ambientes,ambientes_extra,ano_de_construccion,aptos_por_piso,banos,banos_extra,cantidad_de_pisos,casco,...,disposicion,estado,extra,kind,referencia,sobre,tipo_propiedad,title,url,zona
714,NaN,NaN,NaN,False,2016.0,NaN,1.0,False,NaN,NaN,...,al frente,excelente estado,"terraza lavadero,placard en dormitorio,placard...",venta,517b2,avenida,apartamentos,av. italia 2 dormitorios,https://www.infocasas.com.uy/av-italia-2-dormi...,malvin
1259,NaN,NaN,NaN,False,NaN,NaN,1.0,False,NaN,NaN,...,al frente,excelente estado,,venta,94809,rambla,apartamentos,"oportunidad en palermo, cooperativa, dos dormi...",https://www.infocasas.com.uy/oportunidad-en-pa...,palermo
1649,NaN,NaN,NaN,False,NaN,NaN,1.0,False,NaN,NaN,...,al frente,excelente estado,"placard en dormitorio,placard en cocina,patio,...",venta,af582,,apartamentos,excelente apartamento de 2 dormitorios,https://www.infocasas.com.uy/excelente-apartam...,malvin
7417,NaN,NaN,NaN,False,NaN,NaN,1.0,False,NaN,NaN,...,,excelente estado,,venta,0b6c2,,apartamentos,"oportunidad en tres cruces, 3 dormitorios!!",https://www.infocasas.com.uy/oportunidad-en-tr...,tres cruces
9883,NaN,NaN,NaN,False,NaN,NaN,3.0,True,NaN,NaN,...,al frente,excelente estado,"terraza,placard en dormitorio,placard en cocin...",venta,8f105,avenida,apartamentos,"apto, 4 dormitorios, pocitos.",https://www.infocasas.com.uy/apto-4-dormitorio...,pocitos
15001,NaN,NaN,NaN,False,2018.0,NaN,1.0,False,NaN,NaN,...,al frente,a estrenar,"losa radiante,living comedor,garaje,calefaccio...",venta,4fd2a,,apartamentos,doce 22,https://www.infocasas.com.uy/doce-22/185850462?v,pocitos
18045,NaN,NaN,NaN,False,2017.0,NaN,1.0,False,NaN,NaN,...,al frente,a estrenar,"placard en dormitorio,placard en cocina,patio,...",venta,970f4,rambla,apartamentos,"frente al mar, 1 dormitorio, 60 mts, con garaje.",https://www.infocasas.com.uy/frente-al-mar-1-d...,malvin
25537,NaN,NaN,NaN,False,2018.0,NaN,2.0,False,NaN,NaN,...,,a estrenar,"living comedor,lavadero,garaje,calefaccion ind...",venta,a298f,,apartamentos,doce 22,https://www.infocasas.com.uy/doce-22/185850465?v,pocitos nuevo


In [32]:
df_raw.loc[18045, 'url']

'https://www.infocasas.com.uy/frente-al-mar-1-dormitorio-60-mts-con-garaje/185886606?v'

In [23]:
df_selected.describe()

,ano_de_construccion,banos,dormitorios,garajes,gastos_comunes,m2_de_la_terraza,m2_del_terreno,m2_edificados,price,vivienda_social
count,11567.000000,11567.000000,11567.000000,11567.000000,1.156700e+04,11567.000000,11567.000000,11567.000000,1.156700e+04,11567.000000
mean,1989.047981,1.663439,2.325322,0.848102,2.017233e+11,7.740512,130.652870,107.638853,2.618602e+05,0.084897
std,32.872763,0.769398,1.147998,4.633369,1.464612e+13,28.160832,150.619516,92.470602,2.376029e+05,0.278740
min,1060.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,-61.000000,1.000000,1.125000e+04,0.000000
25%,1965.000000,1.000000,2.000000,0.000000,0.000000e+00,0.000000,52.000000,54.000000,1.300000e+05,0.000000
50%,1998.000000,1.000000,2.000000,0.000000,9.500000e+02,0.000000,75.000000,75.000000,1.800000e+05,0.000000
75%,2017.000000,2.000000,3.000000,1.000000,5.000000e+03,5.000000,136.000000,128.000000,3.000000e+05,0.000000
max,2108.000000,3.000000,5.000000,127.000000,1.111111e+15,810.000000,996.000000,930.000000,3.600000e+06,1.000000


In [73]:
#df_selected['m2_del_terreno'] > 1
#df_selected['gastos_comunes'] < 1e5
#df_selected['m2_edificados'] > 20

(
    df_selected.query('m2_del_terreno > 2')
    .query('gastos_comunes < 1e5')
    .query('m2_edificados > 20')
    #.pipe(lambda x: x.describe().transpose())
    .to_csv('../data/ready/num_dataset_houses.csv', index=False)
)


,ano_de_construccion,banos,dormitorios,garajes,gastos_comunes,m2_de_la_terraza,m2_del_terreno,m2_edificados,price,vivienda_social
count,11567.000000,11567.000000,11567.000000,11567.000000,1.156700e+04,11567.000000,11567.000000,11567.000000,1.156700e+04,11567.000000
mean,1989.047981,1.663439,2.325322,0.848102,2.017233e+11,7.740512,130.652870,107.638853,2.618602e+05,0.084897
std,32.872763,0.769398,1.147998,4.633369,1.464612e+13,28.160832,150.619516,92.470602,2.376029e+05,0.278740
min,1060.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,-61.000000,1.000000,1.125000e+04,0.000000
25%,1965.000000,1.000000,2.000000,0.000000,0.000000e+00,0.000000,52.000000,54.000000,1.300000e+05,0.000000
50%,1998.000000,1.000000,2.000000,0.000000,9.500000e+02,0.000000,75.000000,75.000000,1.800000e+05,0.000000
75%,2017.000000,2.000000,3.000000,1.000000,5.000000e+03,5.000000,136.000000,128.000000,3.000000e+05,0.000000
max,2108.000000,3.000000,5.000000,127.000000,1.111111e+15,810.000000,996.000000,930.000000,3.600000e+06,1.000000
